In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2" # Choose which GPUs by checking current use with nvidia-smi
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
## Keras library also provides ResNet101V2 and ResNet50V2. Import them and use it for other experiments. 
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import metrics
import time
import numpy as np
from keras import backend as K



# Check CUDA functionality, restart kernel to change GPUs
gpus = tf.config.list_physical_devices('GPU')
print("*************************************")
print(gpus)
print("*************************************")

# Define function to preprocess images as required by ResNet
def preprocess(images, labels):
    return tf.keras.applications.resnet_v2.preprocess_input(images), labels


#setup train, validation, and test folders
traindir = './train_800_600_simple_comparison_birad'
valdir = './val_800_600_simple_comparison_birad'
testdir = './test_800_600_simple_comparison_birad'
dirName = '800_600'


buffersize = 3
#im_dim = 512
im_dim_x = 800
im_dim_y = 600
batchSizeIntInitial = 10 
batchSizeInt = 6

train = tf.keras.preprocessing.image_dataset_from_directory(
    traindir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)
val = tf.keras.preprocessing.image_dataset_from_directory(
    valdir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)
test = tf.keras.preprocessing.image_dataset_from_directory(
    testdir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)

test_ds = test.map(preprocess)
train_ds = train.map(preprocess)
val_ds = val.map(preprocess)
train_ds = train_ds.prefetch(buffer_size=buffersize)
val_ds = val_ds.prefetch(buffer_size=buffersize)


## set up hyperparameters, such as epochs, learning rates, cutoffs.
epochs = 20
lr = 0.004
cutoff=0.5
start_time = time.time()
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope(): # the entire model needs to be compiled within the scope of the distribution strategy
    cb1 = EarlyStopping(monitor='val_accuracy', patience=4) # define early stopping callback function
    cb2 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=0.00001) # define LR reduction callback function
    opt = keras.optimizers.Adam(learning_rate=lr)
    metr = [metrics.BinaryAccuracy(name='accuracy', threshold=cutoff), metrics.AUC(name='auc'), metrics.Precision(name='precision'),
                metrics.Recall(name='recall')]
#                 tfa.metrics.F1Score(name='f1_score')]
    ptmodel = ResNet50V2(include_top=False, weights='imagenet', classes=2, input_shape=(im_dim_x, im_dim_y, 3), pooling='avg') # compile resnet152v2 with imagenet weights
    ptmodel.trainable = False # freeze layers
    ptmodel.layers[-1].trainable == True

    # un-freeze the BatchNorm layers
    for layer in ptmodel.layers:
        if "BatchNormalization" in layer.__class__.__name__:
            layer.trainable = True

    last_output = ptmodel.output
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(2048, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.15)(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
#    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    # # x = tf.keras.layers.Dropout(0.5, seed=34)(x)
    # x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    model = tf.keras.Model(ptmodel.input, x)
    model.compile(optimizer=opt,
                  loss='BinaryCrossentropy',
                  metrics=metr)

print("---time taken : %s seconds ---" % (time.time() - start_time))
# Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[cb1, cb2])
print("---time taken : %s seconds ---" % (time.time() - start_time))
# Test model
testloss, testaccuracy, testauc, precision, recall = model.evaluate(test_ds)
print('Test accuracy :', testaccuracy)
print('Test AUC :', testauc)

F1 = 2*float(precision)*float(recall)/(float(precision) + float(recall))
print('Test F1 :', F1)
print('Test precision :', precision)
print('Test recall :', recall)

# Model path setup. 
if not os.path.exists("saved_model_resnet50_simple"+dirName):
    os.makedirs("saved_model_resnet50_simple"+dirName+'/')
    
model.save('saved_model_resnet50_simple'+dirName+'/resnet152v2_1')
predicted_probs = np.array([])
true_classes =  np.array([])
IterationChecker = 0
for images, labels in test_ds:
    if IterationChecker == 0:
        predicted_probs = model(images)
        true_classes = labels.numpy()

    IterationChecker += 1

    predicted_probs = np.concatenate([predicted_probs,
                       model(images)])
    true_classes = np.concatenate([true_classes, labels.numpy()])
# Since they are sigmoid outputs, you need to transform them into classes with a threshold, i.e 0.5 here:
predicted_classes = [1 * (x[0]>=cutoff) for x in predicted_probs]
# confusion matrix etc:
conf_matrix = tf.math.confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

predicted_probs=np.squeeze(predicted_probs)
predicted_classes = np.array(predicted_classes)
true_classes=np.squeeze(true_classes)
summedResults = np.stack((predicted_probs,predicted_classes,true_classes), axis = 1)
##Print out statistics which test files are correctly predicted or not. 
np.savetxt("Resnet50_simple_comp_EMBED.csv", summedResults, delimiter=',', header="predicted_probabilty,predicted_classes,true_classes", comments="")

2023-02-01 11:19:55.668543: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-01 11:19:56.392193: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-01 11:19:56.433724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 11:19:56.433988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-01 11:19:56.434052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

*************************************
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
*************************************
Found 1404 files belonging to 2 classes.


2023-02-01 11:19:56.475723: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 11:19:56.725461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 11:19:56.725713: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2023-02-01 11:19:56.725752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1)

Found 452 files belonging to 2 classes.
Found 471 files belonging to 2 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce 

2023-02-01 11:19:59.835546: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}

2023-02-01 11:19:59.845097: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-02-01 11:19:59.864887: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3800000000 Hz


...

234/234 [==============================] - 49s 87ms/step - loss: 0.7815 - accuracy: 0.5328 - auc: 0.5452 - precision: 0.5278 - recall: 0.5837 - val_loss: 0.6945 - val_accuracy: 0.4845 - val_auc: 0.6643 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
234/234 [==============================] - 17s 70ms/step - loss: 0.6202 - accuracy: 0.6467 - auc: 0.7176 - precision: 0.6337 - recall: 0.6881 - val_loss: 0.6509 - val_accuracy: 0.6527 - val_auc: 0.7279 - val_precision: 0.6098 - val_recall: 0.9056
Epoch 3/20
234/234 [==============================] - 17s 71ms/step - loss: 0.5095 - accuracy: 0.7642 - auc: 0.8316 - precision: 0.7317 - recall: 0.8312 - val_loss: 0.7820 - val_accuracy: 0.5819 - val_auc: 0.7051 - val_precision: 0.5526 - val_recall: 0.9914
Epoch 4/20
234/234 [==============================] - 17s 71ms/step - loss: 0.4434 - accuracy: 0.7984 - auc: 0.8743 - precision: 0.7549 - recall: 0.8813 - val_loss: 0.7810 - val_accuracy: 0.6504 - val_auc: 0.7442 - val_precision:

2023-02-01 11:23:36.048770: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:695] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}



79/79 [==============================] - 2s 28ms/step - loss: 0.9766 - accuracy: 0.8110 - auc: 0.8545 - precision: 0.8398 - recall: 0.7886
Test accuracy : 0.8110403418540955
Test AUC : 0.854498565196991
Test F1 : 0.8134171946611464
Test precision : 0.8398268222808838
Test recall : 0.7886179089546204


2023-02-01 11:23:42.931516: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/home/careinfolab/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: saved_model_resnet50_simple800_600/resnet152v2_1/assets
tf.Tensor(
[[188  39]
 [ 53 197]], shape=(2, 2), dtype=int32)
